In [1]:
import requests
import json
import pandas as pd
from connect_db import connect_to_grad_db

In [7]:
resp = requests.get(
    "https://www.trustpilot.com/_next/data/businessunitprofile-consumersite-7193/review/www.teslamotors.com.json",
    )

In [14]:
with open("tp.json", "w") as file:
    json.dump(resp.json(), file)

In [23]:
def get_review_data(
    company_website:str, 
    page:int=1,
    sort:str="recency",
    date="",
    stars="",
):
    BASE_URL = "https://www.trustpilot.com/_next/data/businessunitprofile-consumersite-7193/review/"

    target_url = BASE_URL + company_website + ".json?"

    target_url = f"{target_url}sort={sort}"
    
    if page > 1:
        target_url+=f"&page={page}"
        
    if date:
        target_url+=f"&date={date}"
    
    if stars:
        target_url+=f"&stars={stars}"

    print(target_url)
    resp = requests.get(target_url)

    if not resp.ok:
        print(f"WARNING: something went wrong making requests, status code: {resp.status_code}")
    else:
        return resp.json()

def extract_reviews(raw_reviews):
    extracted = []
    try:
        reviews=raw_reviews["pageProps"]["reviews"]
    
    
        for review in reviews:
            text = review["text"] if review["text"] else ""
            rating = review["rating"] if review["rating"] else None
            title = review["title"] if review["title"] else ""
            rating = review["rating"] if review["rating"] else None
            creation_date = review["dates"]["publishedDate"] if review["dates"]["publishedDate"] else None

            clean_review = {
                "text":text,
                "rating":rating,
                "title":title,
                "creation_date":creation_date
            }

            extracted.append(clean_review)
    except Exception as e:
        print(e)
    
    return extracted
        
def get_reviews(company_website, num_pages=1, start_page=1):
    reviews = []
    for page in range(start_page, start_page+num_pages):
        raw_review = get_review_data(company_website, page=page)
        clean_reviews = extract_reviews(raw_review)
        reviews += clean_reviews
    
    return reviews



In [24]:
extract_reviews(get_review_data("www.norwegian.com", page=3))

https://www.trustpilot.com/_next/data/businessunitprofile-consumersite-7193/review/www.norwegian.com.json?&sort=recency&page=3


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [45]:
reviews = get_reviews("www.norwegian.com", 43)

In [47]:
df = pd.DataFrame(reviews)

In [48]:
df["creation_date"] = pd.to_datetime(df["creation_date"])

In [49]:
df["reviews_for_site"] = "www.norwegian.com"

In [50]:
engine = connect_to_grad_db()
df.to_sql(
    "trustpilot_api",
    engine,
    if_exists="append",
    index=False
)

860